In [1]:
import numpy as np
from utils import *
from utils import *
from dgp import sample_network_chain, get_graph, sample_Y1, agcEffect
import numpy as np


def prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree):
    # 1. Simulate or load data
    adj1 = get_graph(sample_size, min_degree, max_degree, seed=1)
    print(adj1.sum(axis=1).min(), adj1.sum(axis=1).max())

    tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
    rho = np.array([[0,0.1,0.2],
                    [0.1,0,0.1],
                    [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
    nu = np.array([0.1,0,0,0.1,0,0,0.1,0,0]).reshape(3,3)       # shape (3, 3)
    gamma = np.array([-1,2,0.1,-2,0.1,2,0.1,0.1])    # shape (8,)   
    beta = np.array([-1*min_degree,-2*min_degree,2,2,0.1,-1,0.1,2,0.1,0])   # shape (10,)

    res_truth1 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.7,
        R=100,
        burnin_R=100,
        seed=0
    )
    res_truth2 = agcEffect(
        adj1,
        tau, rho, nu, beta,
        treatment_allocation=0.3,
        R=100,
        burnin_R=100,
        seed=0
    )

    ground_truth1 = {}
    ground_truth1['average'] = res_truth1['average']
    ground_truth1['direct'] = res_truth1['direct_effect']
    ground_truth1['indirect'] = res_truth1['psi_0_gamma'] - res_truth2['psi_0_gamma']
    ground_truth1['spillover_effect'] = res_truth1['spillover_effect']
    ground_truth1['psi_1_gamma'] = res_truth1['psi_1_gamma']
    ground_truth1['psi_0_gamma'] = res_truth1['psi_0_gamma']
    ground_truth1['psi_zero'] = res_truth1['psi_zero']

    return adj1, None, None, None, ground_truth1

num_sample = 300
num_burn = 20
n_cpu = 10

In [ ]:
def get_results(min_degree, max_degree, sample_size):
    adj1, _, _, _, ground_truth1 = prepare(num_sample, num_burn, sample_size, n_cpu, min_degree, max_degree)

    res_sum = {}

    for mispec in [None, 'outcome', 'treatment']:
        for model in ['dr_em', 'dr', 'ag']:
            print('Processing model:', model, 'with mispecification:', mispec, f'min_max_{min_degree}_{max_degree}')
            if model == 'ag' and mispec == 'treatment':
                continue
            results1 = np.load(f'run/sim_results/sim_{model}_raw_{sample_size}_{min_degree}_{max_degree}_{0.7}_{mispec}.npy', allow_pickle=True)
            results2 = np.load(f'run/sim_results/sim_{model}_raw_{sample_size}_{min_degree}_{max_degree}_{0.3}_{mispec}.npy', allow_pickle=True)

            if model == 'ag':
                res_ = compute_stats_ag(results1, results2, ground_truth1)
            else:
                res_ = compute_stats(results1, results2, ground_truth1, adj1)
            res_sum[f'model[{model}]_samplesize[{sample_size}]_mdegree[{min_degree}]_maxdegree[{max_degree}_mispec[{mispec}]]'] = res_
                
            
            # write res_ to a txt file by printing it out
            with open(f'run/sim_txt_output/{min_degree}_{max_degree}_{sample_size}.txt', 'a') as f:
                f.write(f'model[{model}]_samplesize[{sample_size}]_mdegree[{min_degree}]_maxdegree[{max_degree}_mispec[{mispec}]]\n')
                f.write(str(res_))
                f.write('\n')

    return res_sum

In [3]:
# (1,2) (2,5) (3, 10)

In [5]:
res_sum_all = {}
sample_size = 800

for min_degree, max_degree in [(2, 8), (2, 10),(4, 10),
                               (1, 2), (2, 5), (3, 10),
                               (1, 4), (1, 8), (1, 10),(2, 6)]:
    res_sum = get_results(min_degree, max_degree, sample_size)
    res_sum_all[(min_degree, max_degree)] = res_sum


# Save the results to a file
import pickle
with open('run/sim_results_v2/res_sum_all.pkl', 'wb') as f:
    pickle.dump(res_sum_all, f)

2 8
Processing model: dr_em with mispecification: None min_max_2_8


100%|██████████| 2000/2000 [02:26<00:00, 13.67it/s]


Processing model: dr with mispecification: None min_max_2_8


100%|██████████| 2000/2000 [02:25<00:00, 13.78it/s]


Processing model: ag with mispecification: None min_max_2_8
Processing model: dr_em with mispecification: outcome min_max_2_8


...

Processing model: ag with mispecification: outcome min_max_2_6
Processing model: dr_em with mispecification: treatment min_max_2_6


100%|██████████| 2000/2000 [02:15<00:00, 14.80it/s]


Processing model: dr with mispecification: treatment min_max_2_6


100%|██████████| 2000/2000 [02:15<00:00, 14.76it/s]

Processing model: ag with mispecification: treatment min_max_2_6
